# PA1: A* Algorithm

In [11]:
# Kindly run the cell below first, containing the helper functions for the code, before this one

# importing numpy to handle the matrices (handling the tiles in 3x3 format)
import numpy as np

for h in range(3):
    # INPUT FIELDS
    # H: which H to use: 1 for Number of Tiles in Wrong Position; 2 for Manhattan Distance; and 3 for Nilsson's Sequence Score
    # txt: name text file name enclosed in quotation marks (ex: "file.txt")
    H = h+1
    txt = "astar_in.txt"
    
    
    # Create the placeholder arrays for OPEN and CLOSED and their corresponding "helper" arrays
    OPEN = []
    f_open_values = [] # to contain the f values of the nodes in OPEN
    g_open_values = [] # to contain the g values of the nodes in OPEN
    h_open_values = [] # to contain the h values of the nodes in OPEN
    p_open_values = [] # to contain the p values of the nodes in OPEN (for Nilsson's Sequence Score)
    s_open_values = [] # to contain the s values of the nodes in OPEN (for Nilsson's Sequence Score)
    open_pointers = [] # to contain the pointers of the nodes in OPEN
    
    CLOSED = []
    f_closed_values = [] # to contain the f values of the nodes in CLOSED
    g_closed_values = [] # to contain the g values of the nodes in CLOSED
    h_closed_values = [] # to contain the h values of the nodes in CLOSED
    p_closed_values = [] # to contain the p values of the nodes in CLOSED (for Nilsson's Sequence Score)
    s_closed_values = [] # to contain the s values of the nodes in CLOSED (for Nilsson's Sequence Score)
    closed_pointers = [] # to contain the pointers of the nodes in CLOSED
    
    # To record the number of nodes generated
    search_cost = 0
    
    # Retrieve the start and goal states
    initial = txt_to_array(txt)
    start = initial[0]
    goal = initial[1]
    
    # PSEUDO-CODE STEP 1
    # Assign the start state as the initial value in OPEN and calculate initial values (g, h, and f)
    if H == 1:
        h_0 = heuristic_one(start,goal)
    elif H == 2:
        h_0 = heuristic_two(start,goal)
    elif H == 3:
        h_0 = heuristic_three(start,goal)
        p_0 = heuristic_two(start,goal)
        s_0 = (h_0 - p_0)/3
        p_open_values.append(p_0)
        s_open_values.append(s_0)
    OPEN.append(start)
    h_open_values.append(h_0)
    g_open_values.append(0)
    f_open_values.append(0+h_0)
    open_pointers.append(np.zeros_like(start))
    
    # PSEUDO-CODE STEP 2
    # Continue while OPEN is not empty
    while len(OPEN) != 0:
    
        # PSEUDO-CODE STEP 3
        # Find the minimum f in the OPEN
        t = min(f_open_values)
        n_add = f_open_values.index(t)
        t_g = g_open_values[n_add]
        t_h = h_open_values[n_add]
        n = OPEN[n_add]
        n_pointer = open_pointers[n_add]
        
        # Assign to CLOSED the state in OPEN with the minimum f
        CLOSED.append(n)
        f_closed_values.append(t)
        g_closed_values.append(t_g)
        h_closed_values.append(t_h)
        closed_pointers.append(n_pointer)
        
        # Remove the OPEN with minimum f
        del OPEN[n_add]
        del f_open_values[n_add]
        del g_open_values[n_add]
        del h_open_values[n_add]
        del open_pointers[n_add]
    
        if H == 3:
            t_p = p_open_values[n_add]
            t_s = s_open_values[n_add]
            p_closed_values.append(t_p)
            s_closed_values.append(t_s)
            del p_open_values[n_add]
            del s_open_values[n_add]
    
        # PSEUDO-CODE STEP 4
        # Check if the minimum f in OPEN (or n) is the goal    
        if np.all(n == goal):
            
            # Tracing the solution path
            if H == 3:
                solution = trace(CLOSED, closed_pointers, f_closed_values, g_closed_values, h_closed_values, p_closed_values, s_closed_values)
                solution_path = solution[0]
                f_values_path = solution[1]
                g_values_path = solution[2]
                h_values_path = solution[3]
                p_values_path = solution[4]
                s_values_path = solution[5]
            else:
                solution = trace(CLOSED, closed_pointers, f_closed_values, g_closed_values, h_closed_values)
                solution_path = solution[0]
                f_values_path = solution[1]
                g_values_path = solution[2]
                h_values_path = solution[3]
            
            break
                    
        else: 
            
            # PSEUDO-CODE STEP 5
            # Find the successors of n and its corresponding f values
            nexts = successors(n)
            search_cost += len(nexts)
            
            if len(nexts) == 0:
                continue
                
            else:
    
                if H == 1:
                    nexts_h = np.array(heuristic_one(nexts,goal))
                elif H == 2:
                    nexts_h = np.array(heuristic_two(nexts,goal))
                elif H == 3:
                    nexts_h = np.array(heuristic_three(nexts,goal))
                    nexts_p = np.array(heuristic_two(nexts,goal))
                    nexts_s = (nexts_h - nexts_p)/3
                
                nexts_g = g_value(n, CLOSED, closed_pointers)
                nexts_f = nexts_g + nexts_h
                
                new = np.zeros_like(nexts_f)
                remove = []
    
                # PSEUDO-CODE STEPS 6 AND 7
                # Check if the successors are already in OPEN
                for o in range(len(OPEN)):
                    for x in range(len(nexts)):
                        if np.all(nexts[x] == OPEN[o]):
                            new[x] += 1
                            # If they are in OPEN and the new calculated f values are lower, replace the ones in the record
                            if nexts_f[x] < f_open_values[o]:
                                f_open_values[o] = nexts_f[x]
                                g_open_values[o] = nexts_g
                                h_open_values[o] = nexts_h[x]
                                open_pointers[o] = n
                                if H == 3:
                                    p_open_values[o] = nexts_p[x]
                                    s_open_values[o] = nexts_s[x]
                                
                # Check if the successors are already in CLOSED        
                for c in range(len(CLOSED)):
                    for x in range(len(nexts)):
                        if np.all(nexts[x] == CLOSED[c]):
                            new[x] += 1
                            # If they are in CLOSED and have lower new calculated values, put them in the OPEN
                            if nexts_f[x] < f_closed_values[c]:
                                OPEN.append(CLOSED[c])
                                f_open_values.append(nexts_f[x])
                                g_open_values.append(nexts_g)
                                h_open_values.append(nexts_h[x])
                                open_pointers.append(n)
                                # record the index of CLOSED element to be removed
                                remove.append(c)
                                if H == 3:
                                    p_open_values.append(nexts_p[x])
                                    s_open_values.append(nexts_s[x])
            
                # Remove from CLOSED those who were moved to OPEN
                for r in sorted(remove, reverse = True):
                    del CLOSED[r]
                    del f_closed_values[r]
                    del g_closed_values[r]
                    del h_closed_values[r]
                    del closed_pointers[r]
                    if H == 3:
                        del p_closed_values[r]
                        del s_closed_values[r]
            
                # Add to OPEN those successors that are not on either OPEN or CLOSED
                for x in range(len(nexts)):
                    if new[x] == 0:
                        OPEN.append(nexts[x])
                        f_open_values.append(nexts_f[x])
                        g_open_values.append(nexts_g)
                        h_open_values.append(nexts_h[x])
                        open_pointers.append(n)
                        if H == 3:
                            p_open_values.append(nexts_p[x])
                            s_open_values.append(nexts_s[x])
    
                # PSEUDO-CODE STEP 8
                # End of one while loop, start again
    
    
    if len(OPEN) == 0: # exit with failure
        print("No solution found!")
    elif H == 3: # for Nilsson's Sequence Score
        print("\n**********\nH3: Nilsson's Sequence Score\n")
        print(f"Search Cost (number of nodes generated): {search_cost}\n")
        print("Answer format:\n [State number]: [State], f(n) = [g(n)] + [h(n)] (from [P(n)] + 3*[S(n)]) = [f(n)]")
        for c in range(len(solution_path)):
            print(f"{c}:\n {solution_path[c]}, f({c}) = {g_values_path[c]} + {h_values_path[c]} (from {p_values_path[c]} + 3*{s_values_path[c]}) = {f_values_path[c]}")
    elif H == 2: # for the Manhattan distance
        print("\n**********\nH2: Manhattan Distance\n")
        print(f"Search Cost (number of nodes generated): {search_cost}\n")
        print("Answer format:\n [State number]: [State], f(n) = [g(n)] + [h(n)] = [f(n)]")
        for c in range(len(solution_path)):
            print(f"{c}:\n {solution_path[c]}, f({c}) = {g_values_path[c]} + {h_values_path[c]} = {f_values_path[c]}")
    elif H == 1: # for the number of wrong positions
        print("\n**********\nH1: Number of Tiles in Wrong Position\n")
        print(f"Search Cost (number of nodes generated): {search_cost}\n")
        print("Answer format:\n [State number]: [State], f(n) = [g(n)] + [h(n)] = [f(n)]")
        for c in range(len(solution_path)):
            print(f"{c}:\n {solution_path[c]}, f({c}) = {g_values_path[c]} + {h_values_path[c]} = {f_values_path[c]}")
    


# END OF THE MAIN LINE OF CODES


**********
H1: Number of Tiles in Wrong Position

Search Cost (number of nodes generated): 4782

Answer format:
 [State number]: [State], f(n) = [g(n)] + [h(n)] = [f(n)]
0:
 [['2' '1' '6']
 ['4' '*' '8']
 ['7' '5' '3']], f(0) = 0 + 7 = 7
1:
 [['2' '1' '6']
 ['4' '8' '*']
 ['7' '5' '3']], f(1) = 1 + 7 = 8
2:
 [['2' '1' '*']
 ['4' '8' '6']
 ['7' '5' '3']], f(2) = 2 + 7 = 9
3:
 [['2' '*' '1']
 ['4' '8' '6']
 ['7' '5' '3']], f(3) = 3 + 7 = 10
4:
 [['2' '8' '1']
 ['4' '*' '6']
 ['7' '5' '3']], f(4) = 4 + 7 = 11
5:
 [['2' '8' '1']
 ['4' '6' '*']
 ['7' '5' '3']], f(5) = 5 + 7 = 12
6:
 [['2' '8' '1']
 ['4' '6' '3']
 ['7' '5' '*']], f(6) = 6 + 7 = 13
7:
 [['2' '8' '1']
 ['4' '6' '3']
 ['7' '*' '5']], f(7) = 7 + 6 = 13
8:
 [['2' '8' '1']
 ['4' '*' '3']
 ['7' '6' '5']], f(8) = 8 + 5 = 13
9:
 [['2' '8' '1']
 ['*' '4' '3']
 ['7' '6' '5']], f(9) = 9 + 5 = 14
10:
 [['*' '8' '1']
 ['2' '4' '3']
 ['7' '6' '5']], f(10) = 10 + 5 = 15
11:
 [['8' '*' '1']
 ['2' '4' '3']
 ['7' '6' '5']], f(11) = 11 + 5 = 1

In [3]:
# Run this cell first
#***7 helper functions are found below***

# FUNCTION TO RETRIEVE THE START & GOAL STATES, AND CONVERT THEM INTO ARRAYS
# takes in the text file name as the input

def txt_to_array(text_file):
    
    # Retrieving text file, and pulling start and goal states
    file = open(text_file)
    content = file.readlines()
    file.close()
    
    start_initial = []
    goal_initial = []
    for i in range(len(content)):
        if content[i] == "start\n":
            for s in range(3):
                start_initial.append(content[s+1])
        elif content[i] == "goal\n":
            for g in range(3):
                goal_initial.append(content[i+g+1])
    
    # Removing unnecessary characters in both the start and goal states
    start_list = []
    for i in range(len(start_initial)):
        for char in start_initial[i]:
            if char.isdigit():
                start_list.append(char)
            elif char == "*":
                start_list.append(char)
    
    goal_list = []
    for i in range(len(goal_initial)):
        for char in goal_initial[i]:
            if char.isdigit():
                goal_list.append(char)
            elif char == "*":
                goal_list.append(char)
    
    # Converting lists into arrays
    start = np.array(start_list).reshape(3,3)
    goal = np.array(goal_list).reshape(3,3)
    output = [start, goal]
    
    return output



# FUNCTION TO GENERATE ALL THE SUCCESSORS
# takes in the 3x3 parent node or state as the input

def successors(first): 

    # Create the placeholders for the possible moves, and extract the position of the "*" tile
    star = np.argwhere(first == "*")
    x_moves = []
    y_moves = []

    # Series of IFS to append the possible moves
    # Currently built for 3x3 or 8-puzzle as the values being compared are 0 and 2
    if star[0][1] > 0: # left
        x_moves.append(star[0][1] -1) 
        y_moves.append(star[0][0]) 
    if star[0][1] < 2: # right
        x_moves.append(star[0][1] +1) 
        y_moves.append(star[0][0]) 
    if star[0][0] > 0: # up
        x_moves.append(star[0][1])  
        y_moves.append(star[0][0] -1)
    if star[0][0] < 2: # down
        x_moves.append(star[0][1])  
        y_moves.append(star[0][0] +1)
    
    current = []
    new_states = []
    for n in range(len(y_moves)):
        current = first.copy()
        # assign the number to *'s old position
        current[star[0][0]][star[0][1]] = current[y_moves[n]][x_moves[n]]    
        # assign * to new position
        current[y_moves[n]][x_moves[n]] = "*"                                
        # append new state to array
        new_states.append(current)                                           
    
    new_states = np.array(new_states)

    return new_states



# FUNCTION FOR H1: TO FIND THE NUMBER OF TILES IN WRONG POSITION
# takes in the a new state (new_state), and the goal state (target) as inputs

def heuristic_one(new_state, target):

    # Placeholders
    current = []
    wrongs = []
    wrong = 0

    # For start state and possible single new state
    if new_state.shape == (3,3):
        for i in range(3):
            for j in range(3):
                if new_state[i][j].isdigit() and new_state[i][j] != target[i][j]:
                    wrong += 1
                else:
                    wrong += 0
                    
        wrongs = np.array(wrong)

    # For multiple new states
    else:
        for a in range(len(new_state)):
            current = new_state[a].copy()
            for i in range(3):
                for j in range(3):
                    if current[i][j].isdigit() and current[i][j] != target[i][j]:
                        wrong += 1
                    else:
                        wrong += 0
                        
            wrongs.append(wrong)
            wrong = 0
                
    wrongs = np.array(wrongs)

    return wrongs



# FUNCTION FOR H2: TO FIND THE MANHATTAN DISTANCE

def heuristic_two(new, target):

    # Placeholders
    pos = np.array([[0, 0]])
    distance=[]
    distances=[]
    
    # Append to placeholder (pos) the position of each tile in the goal state (in order)
    for r in range(8):
        pos_0 = np.argwhere(target == f"{r+1}")
        pos = np.append(pos,pos_0, axis=0)
    
    # Find the Manhattan distance of each tile from the goal position and append to placeholder (distance)
    # For start state and possible single new state
    if new.shape == (3,3):
        for i in range(3):
            for j in range(3):
                if new[i][j].isdigit():
                    dist = sum(abs(pos[int(new[i][j])] - [i, j]))
                    distance.append(dist)
        
        # Find the total Manhattan distance for the state
        distances = sum(distance)
    
    # For multiple new states
    else:
        for a in range(len(new)):
            current = new[a].copy()
            for i in range(3):
                for j in range(3):
                    if current[i][j].isdigit():
                        dist = sum(abs(pos[int(current[i][j])] - [i, j]))
                        distance.append(dist)
    
            # Find the total Manhattan distance for the state
            distance = sum(distance)
            distances.append(distance)
            distance=[]
    
    return np.array(distances)



# FUNCTION FOR H3: TO FIND THE NILLSON'S SEQUENCE SCORE

def heuristic_three(new, target):

    # Calculate the Manhattan distance
    P = np.array(heuristic_two(new, target))
    
    score = []

    # List containing the indeces starting at the upper left corner, then going clockwise
    # will only work for goal states that have the blank til (*) in the middle
    add = [[0, 0],
           [0, 1],
           [0, 2],
           [1, 2],
           [2, 2],
           [2, 1],
           [2, 0],
           [1, 0],
           [0, 0]]

    if new.shape == (3,3):
        # Add score if the middle tile is a number
        if new[1][1].isdigit():
            score.append(1)
        for a in range(len(add)-1):
            s = new[add[a][0]][add[a][1]]
            if s.isdigit():
                g_v = np.array(np.argwhere(target == f"{s}")[0])
                g = add.index([g_v[0], g_v[1]])
                g_c = target[add[g+1][0]][add[g+1][1]]
                s_c = new[add[a+1][0]][add[a+1][1]]
                if g_c != s_c:
                    score.append(2)
        S = sum(score)

    else:
        S = []
        for n in range(len(new)):
            current = new[n].copy()
            # Add score if the middle tile is a number
            if current[1][1].isdigit():
                score.append(1)
            for a in range(len(add)-1):
                s = current[add[a][0]][add[a][1]]
                if s.isdigit():
                    g_v = np.array(np.argwhere(target == f"{s}")[0])
                    g = add.index([g_v[0], g_v[1]])
                    g_c = target[add[g+1][0]][add[g+1][1]]
                    s_c = current[add[a+1][0]][add[a+1][1]]
                    if g_c != s_c:
                        score.append(2)
            S.append(sum(score))
            score=[]

    H = P + 3*np.array(S)

    return H



# FUNCTION to get the g value
# takes in the current

def g_value(child, children, parents):
    
    trace = []
    current = child
    last = np.zeros_like(current)

    while not np.all(current == last):

        for c in range(len(children)):
            if np.all(children[c] == current):
                trace.append(parents[c])
                current = parents[c]

    r = len(trace)
    
    return r



# FUNCTION to get the solution path

def trace(children, parents, val1, val2, val3, val4 = [], val5 = []):
    
    trace = []
    trace.append(children[-1])
    trace.append(parents[-1])

    value1 = []
    value1.append(val1[-1])
    value2 = []
    value2.append(val2[-1])
    value3 = []
    value3.append(val3[-1])
    if len(val4) > 0 and len(val5) > 0:
        value4 = []
        value4.append(val4[-1])
        value5 = []
        value5.append(val5[-1])

    
    current = parents[-1]
    last = np.zeros_like(current)

    while not np.all(current == last):

        for c in range(len(children)):
            if np.all(children[c] == current):
                trace.append(parents[c])
                value1.append(val1[c])
                value2.append(val2[c])
                value3.append(val3[c])
                current = parents[c]
                if len(val4) > 0 and len(val5) > 0:
                    value4.append(val4[c])
                    value5.append(val5[c])

    r = len(trace)
    del trace[r-1]
    trace.reverse()
    value1.reverse()
    value2.reverse()
    value3.reverse()
    if len(val4) > 0 and len(val5) > 0:
        value4.reverse()
        value5.reverse()

    if len(val4) > 0 and len(val5) > 0:
        output = [trace, value1, value2, value3, value4, value5]
    else:
        output = [trace, value1, value2, value3]
    
    return output